# IBM Data Science Capstone - Week 4 Task
### Applied Data Science Capstone by IBM - Coursera

## Table of contents
* [1. Introduction: Business Problem](#introduction)
* [2. Data](#data)

## 1. Introduction: Business Problem

In this project, we will evaluate the best place to **open a Art Gallery in the city of São Paulo**, located in **Brazil, South America**.  The city's metropolitan area, the Greater São Paulo, ranks as the most populous in Brazil and the 12th most populous on Earth. 

Having the largest economy by GDP in Latin America and the Southern Hemisphere, the city has the **11th largest GDP in the world**, representing alone 10.7% of all Brazilian GDP and 36% of the production of goods and services in the state of São Paulo, being home to 63% of established multinationals in Brazil, and has been responsible for 28% of the national scientific production.

Then, it is quite obvious that there are many museums and art galleries in every corner of the town. However, as many cities of the developing world, it is an unequal city, having many different borough standarts. There are very expensive and very poor neighborhoods. Thereby, our analysis will be based on popularity of neighborhoods: **the boroughs with more places relation to art have more chance to be good locations for our stakeholders invest**.

## 2. Data

### 2.1 City Hall Data

We will start getting the coordinates of all the boroughs of São Paulo, that are **avaiable in the City Hall website**, and create a dataframe with them.

In [1]:
import pandas as pd
import numpy as np
import lxml
#link = "https://pt.wikipedia.org/wiki/Coordenadas_Geogr%C3%A1ficas_dos_Bairros_de_S%C3%A3o_Paulo_-_SP#Bairros_e_Coordenadas"
#wikitables = pd.read_html(link, header=0)
#wikitables_df = pd.DataFrame(wikitables) 
#sp = wikitables_df.iloc[0,0]
#sp[['Latitude', 'Longitude']] = sp[['Latitude', 'Longitude']]/1000000
sp = pd.read_csv("Borough_SP.csv")
sp.rename(columns={' Longitude':'Longitude'}, inplace=True)
sp.head(5)

,Borough,Latitude,Longitude
0,Sé Bela Vista,-23.559353,-46.647325
1,Bom Retiro,-23.525558,-46.640975
2,Cambuci,-23.564380,-46.621792
3,Consolação,-23.551929,-46.655807
4,Liberdade,-23.559745,-46.635536


Also, lets see the shape of the data. We have 94 boroughs, that is a very good number.

In [2]:
sp.shape

(94, 3)

Now, lets visualize the data in a map, generated with folium.

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [4]:
import folium

# Find the center of all the locations and prepare the folium map
df1 = sp

center_lat= -23.550274 #center around Sé, the center of the city
center_long= -46.633944

venues_map = folium.Map(location=[center_lat, center_long], zoom_start=11)

df2=df1.head(80)
for lat, lng, label in zip(df2.Latitude, df2.Longitude, df2.Borough):
    #print(lat,lng,label)
    folium.features.CircleMarker(
            [lat, lng],
            radius=7,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
    ).add_to(venues_map)
venues_map

Nice! The data is very spread, wich indicates we might have many different styles of neighborhoods. Now we may start to analise the data in Foursquare.

### 2.2 Foursquare Data

The purpose of the data acquired with Foursquare is to analyse the venues (of all sorts) in each neighborhood. After, we will segregate only the venues related to art, applying filter in the dataframe created.

First, we need to initialize our credentials of Foursquare.

In [5]:
CLIENT_ID = 'NBWX43X25ARRKGVM0PG30HQWLX5VJPGYLQH1EVSUZXAK2J3Z' # your Foursquare ID
CLIENT_SECRET = 'FGZECG33UKC4UU42B3PD3CXUDWQ4HCDPTIDQYNK2JWAM5QYM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NBWX43X25ARRKGVM0PG30HQWLX5VJPGYLQH1EVSUZXAK2J3Z
CLIENT_SECRET:FGZECG33UKC4UU42B3PD3CXUDWQ4HCDPTIDQYNK2JWAM5QYM


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
sp_venues = getNearbyVenues(names=sp['Borough'],
                                   latitudes=sp['Latitude'],
                                   longitudes=sp['Longitude']
                                  )

Sé Bela Vista
Bom Retiro
Cambuci
Consolação
Liberdade
República
Santa Cecília
Sé
Aricanduva
Carrão
Vila Formosa
Cidade Tiradentes
Ermelino Matarazzo
Ponte Rasa
Guaianases
Lajeado
Itaim Paulista
Vila Curuçá
Itaquera
Cidade Líder
José Bonifácio
Parque do Carmo
Mooca Água Rasa
Belém
Brás
Moóca
Pari
Tatuapé
Penha
Artur Alvim
Cangaíba
Vila Matilde
São Mateus
São Rafael
São Miguel 
Jardim Helena
Vila Jacuí
Sapopemba
Vila Prudente
São Lucas
Casa Verde
Cachoeirinha
Limão
Brasilândia
 Freguesia do Ó
Jaçanã
Tremembé
Perus
Anhanguera
Pirituba
Jaraguá
São Domingos
Santana
Tucuruvi
Mandaqui
Vila Maria
Vila Guilherme
Vila Medeiros
Butantã
Morumbi
Raposo Tavares
Rio Pequeno
Vila Sônia
Lapa
Barra Funda
Jaguara
Jaguaré
Perdizes
Vila Leopoldina
Pinheiros
Alto de Pinheiros
Itaim Bibi
Jardim Paulista
Campo Limpo
Capão Redondo
Vila Andrade
Grajaú
Socorro
Cidade Ademar
Pedreira
Ipiranga
Sacomã
Jabaquara
M'Boi Mirim
Jardim Ângela
Jardim São Luís
Parelheiros
Marsilac
Santo Amaro
Campo Belo
Campo Grande
Moema


In [8]:
print(sp_venues.shape)
sp_venues.head()

(2813, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sé Bela Vista,-23.559353,-46.647325,Templo da Carne - Marcos Bassi,-23.559061,-46.646452,Steakhouse
1,Sé Bela Vista,-23.559353,-46.647325,Basilicata,-23.558480,-46.646500,Bakery
2,Sé Bela Vista,-23.559353,-46.647325,Pousada dos Franceses,-23.559094,-46.647991,Hostel
3,Sé Bela Vista,-23.559353,-46.647325,Miguel Giannini,-23.558868,-46.647209,Optical Shop
4,Sé Bela Vista,-23.559353,-46.647325,Academia Gaviões,-23.560419,-46.646341,Gym / Fitness Center


In [9]:
# one hot encoding
sp_onehot = pd.get_dummies(sp_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sp_onehot['Neighborhood'] = sp_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sp_onehot.columns[-1]] + list(sp_onehot.columns[:-1])
sp_onehot = sp_onehot[fixed_columns]

sp_onehot.head()

,Neighborhood,Acai House,Accessories Store,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baiano Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Borek Place,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empada House,Empanada Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mental Health Office,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Newsstand,Nightclub,Northeastern Brazilian Restaurant,Northern Brazilian Restaurant,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Samba School,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southeastern Brazilian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Stoop Sale,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Veterinaria

In [10]:
sp_grouped = sp_onehot.groupby('Neighborhood').mean().reset_index()
sp_grouped.head(5)

,Neighborhood,Acai House,Accessories Store,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baiano Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Borek Place,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empada House,Empanada Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mental Health Office,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Newsstand,Nightclub,Northeastern Brazilian Restaurant,Northern Brazilian Restaurant,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Samba School,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southeastern Brazilian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Stoop Sale,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Veterinaria

Ok, now we have for every neighborhood of the city its correspondent mean value of all the venues. The next step is to filter the dataframe which venues related to art. However since the public that would go to an art gallery would also visit other cultural venues, such as: museums, music auditoriums, theater, cultural centers, dance studios, piano bars, libraries, bookshops etc.

Another thing that is very important in visiting cultural places is the availability of public transportation, thus, bus and train stations will be also attributes of our dataframe.

Then, we must filter the table above to give us only the data related to cultural live. In a brief analysis, we can do it, selecting only the columns of interest.

In [11]:
sp_art = sp_grouped[['Neighborhood','Art Gallery','Art Museum','Art Studio','Arts & Crafts Store','Arts & Entertainment','Auditorium',
                     'Antique Shop','Bookstore','Bus Station','Bus Stop','Camera Store','Circus','College Bookstore','College Theater',
                     'Community College','Concert Hall','Cultural Center','Dance Studio','Film Studio','General Entertainment',
                     'History Museum','Historic Site','Indie Movie Theater','Jazz Club','Library','Movie Theater','Museum','Music School',
                     'Music Store','Music Venue','Piano Bar','Public Art','Record Shop','Recording Studio','Rock Club','Science Museum',
                     'Street Art','Theater','Train Station'
                    ]]
sp_art.head(5)

,Neighborhood,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Auditorium,Antique Shop,Bookstore,Bus Station,Bus Stop,Camera Store,Circus,College Bookstore,College Theater,Community College,Concert Hall,Cultural Center,Dance Studio,Film Studio,General Entertainment,History Museum,Historic Site,Indie Movie Theater,Jazz Club,Library,Movie Theater,Museum,Music School,Music Store,Music Venue,Piano Bar,Public Art,Record Shop,Recording Studio,Rock Club,Science Museum,Street Art,Theater,Train Station
0,Alto de Pinheiros,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.045455,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.000,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.045455
1,Anhanguera,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.000,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000
2,Aricanduva,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.000,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000
3,Artur Alvim,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.000,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000
4,Barra Funda,0.0,0.0,0.0,0.0125,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0125,0.000000,0.0,0.0,0.0,0.0,0.0125,0.0,0.0,0.0125,0.025,0.0,0.0,0.025,0.0,0.0125,0.0,0.0,0.0,0.0,0.0,0.0375,0.000000


Now, we have a table that show us the relation of every São Paulo neighborhoods and all the categories related to the arts. In the next section, we will start to work with this data and cluster it!